# Week 09: Exercise 1

## Database

- JHU data on covid cases/death in US Counties
- Daily update in September 2020
- Database is normalized
  - tables for county and state
- Other tables exist
  - `demography` 
  - `election_2016`

In [1]:
!wget https://www.dropbox.com/s/wkhjoah228gmhtl/covid.sqlite?dl=1 -O covid.sqlite

--2020-12-01 14:26:14--  https://www.dropbox.com/s/wkhjoah228gmhtl/covid.sqlite?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.65.1, 2620:100:6021:1::a27d:4101
Connecting to www.dropbox.com (www.dropbox.com)|162.125.65.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/dl/wkhjoah228gmhtl/covid.sqlite [following]
--2020-12-01 14:26:15--  https://www.dropbox.com/s/dl/wkhjoah228gmhtl/covid.sqlite
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucc3010c16d1e54d42e0081d3a0d.dl.dropboxusercontent.com/cd/0/get/BEMYeCvx9ULFk1N4n-dWrwC5NOrgcbZJ3RnsrwJUnSih8zQDZR2N1074IG03nve0iHKCVb1Hgm5tuEPmBeXKxU9f_30oQphigNSxv1fIVcPELsb5-fstUft5ILbKlLSNjPY/file?dl=1# [following]
--2020-12-01 14:26:15--  https://ucc3010c16d1e54d42e0081d3a0d.dl.dropboxusercontent.com/cd/0/get/BEMYeCvx9ULFk1N4n-dWrwC5NOrgcbZJ3RnsrwJUnSih8zQDZR2N1074IG03nve0iHKCVb1Hgm5tuEPmBeXKxU9f_30oQphigNSxv1fIVcPEL

## Import packages

In [2]:
import pandas as pd
import sqlite3

## Create a connection object

In [3]:
con_cov = sqlite3.connect('covid.sqlite')

## Check tables

In [33]:
pd.read_sql_query("SELECT * FROM sqlite_master WHERE type='table'", con_cov)

,type,name,tbl_name,rootpage,sql
0,table,covid,covid,2,"CREATE TABLE ""covid"" (\n""fips"" INTEGER,\n ""da..."
1,table,county,county,940,"CREATE TABLE ""county"" (\n""fips"" INTEGER,\n ""c..."
2,table,state,state,968,"CREATE TABLE ""state"" (\n""state_code"" TEXT,\n ..."
3,table,demography,demography,970,"CREATE TABLE ""demography"" (\n""fips"" INTEGER,\n..."
4,table,election_2016,election_2016,1174,"CREATE TABLE ""election_2016"" (\n""fips"" INTEGER..."
5,table,sep30_table,sep30_table,1205,"CREATE TABLE ""sep30_table"" (\n""index"" INTEGER,..."


In [5]:
pd.read_sql_query("SELECT * FROM covid LIMIT 5", con_cov)

,fips,date,cases,deaths
0,1001,2020-09-01,1354,23
1,1001,2020-09-02,1345,23
2,1001,2020-09-03,1349,23
3,1001,2020-09-04,1355,23
4,1001,2020-09-05,1371,23


In [6]:
pd.read_sql_query("SELECT * FROM county LIMIT 5", con_cov)

,fips,county,state_code
0,1001,Autauga,AL
1,1003,Baldwin,AL
2,1005,Barbour,AL
3,1007,Bibb,AL
4,1009,Blount,AL


In [7]:
pd.read_sql_query("SELECT * FROM state LIMIT 5", con_cov)

,state_code,state
0,AL,Alabama
1,AK,Alaska
2,AZ,Arizona
3,AR,Arkansas
4,CA,California


In [10]:
pd.read_sql_query("SELECT * FROM demography LIMIT 5", con_cov)

,fips,TotalPop,Men,Women,Hispanic,White,Black,Native,Asian,Pacific,VotingAgeCitizen,Income,IncomeErr,IncomePerCap,IncomePerCapErr,Poverty,ChildPoverty,Professional,Service,Office,Construction,Production,Drive,Carpool,Transit,Walk,OtherTransp,WorkAtHome,MeanCommute,Employed,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Unemployment
0,1001,55036,26899,28137,2.7,75.4,18.9,0.3,0.9,0.0,41016,55317,2838,27824,2024,13.7,20.1,35.3,18.0,23.2,8.1,15.4,86.0,9.6,0.1,0.6,1.3,2.5,25.8,24112,74.1,20.2,5.6,0.1,5.2
1,1003,203360,99527,103833,4.4,83.1,9.5,0.8,0.7,0.0,155376,52562,1348,29364,735,11.8,16.1,35.7,18.2,25.6,9.7,10.8,84.7,7.6,0.1,0.8,1.1,5.6,27.0,89527,80.7,12.9,6.3,0.1,5.5
2,1005,26201,13976,12225,4.2,45.7,47.8,0.2,0.6,0.0,20269,33368,2551,17561,798,27.2,44.9,25.0,16.8,22.6,11.5,24.1,83.4,11.1,0.3,2.2,1.7,1.3,23.4,8878,74.1,19.1,6.5,0.3,12.4
3,1007,22580,12251,10329,2.4,74.6,22.0,0.4,0.0,0.0,17662,43404,3431,20911,1889,15.2,26.6,24.4,17.6,19.7,15.9,22.4,86.4,9.5,0.7,0.3,1.7,1.5,30.0,8171,76.0,17.4,6.3,0.3,8.2
4,1009,57667,28490,29177,9.0,87.4,1.5,0.3,0.1,0.0,42513,47412,2630,22021,850,15.6,25.4,28.5,12.9,23.3,15.8,19.5,86.8,10.2,0.1,0.4,0.4,2.1,35.0,21380,83.9,11.9,4.0,0.1,4.9


### Combine covid and county

In [17]:
pd.read_sql_query('''
SELECT a.*, b.county, b.state_code 
  FROM covid AS a 
  INNER JOIN county AS b 
  ON a.fips = b.fips
''', con_cov)

,fips,date,cases,deaths,county,state_code
0,1001,2020-09-01,1354,23,Autauga,AL
1,1001,2020-09-02,1345,23,Autauga,AL
2,1001,2020-09-03,1349,23,Autauga,AL
3,1001,2020-09-04,1355,23,Autauga,AL
4,1001,2020-09-05,1371,23,Autauga,AL
...,...,...,...,...,...,...
97225,56045,2020-09-26,36,0,Weston,WY
97226,56045,2020-09-27,36,0,Weston,WY
97227,56045,2020-09-28,36,0,Weston,WY
97228,56045,2020-09-29,37,0,Weston,WY


### Combine covid, county, and state

In [9]:
pd.read_sql_query('''
SELECT a.*, b.county, b.state_code 
  FROM covid AS a 
  INNER JOIN county AS b 
  ON a.fips = b.fips
  INNER JOIN state as c 
  ON b.state_code = c.state_code''', con_cov)

,fips,date,cases,deaths,county,state_code
0,1001,2020-09-01,1354,23,Autauga,AL
1,1001,2020-09-02,1345,23,Autauga,AL
2,1001,2020-09-03,1349,23,Autauga,AL
3,1001,2020-09-04,1355,23,Autauga,AL
4,1001,2020-09-05,1371,23,Autauga,AL
...,...,...,...,...,...,...
97225,56045,2020-09-26,36,0,Weston,WY
97226,56045,2020-09-27,36,0,Weston,WY
97227,56045,2020-09-28,36,0,Weston,WY
97228,56045,2020-09-29,37,0,Weston,WY


# Replicate the work in Assignment 1 using SQL queries

In this exercise, you will do the same work you have done with SQL queries

## Mergeing with the demography data


### Check the contents of demography table


- Merge covid, county, state, and demography tables
  - select the data on 30 September
- Calculate the deaths/cases per 1000 population in the merged table
- Create a new pandas DataFrame from query
- Dump the content of DataFrame as a table (named `sep30_table` in this database) using `.to_sql`


In [43]:
df_sep30 = pd.read_sql_query('''
SELECT a.*, b.county, b.state_code , c.state, d.*, 
  (deaths + 0.0) / TotalPop * 1000 AS death_per_1000,
  (cases + 0.0) / TotalPop * 1000 AS cases_per_1000
  FROM covid AS a 
  INNER JOIN county AS b 
  ON a.fips = b.fips
  INNER JOIN state AS c 
  ON b.state_code = c.state_code
  INNER JOIN demography AS d
  ON a.fips = d.fips
  WHERE date = "2020-09-30"''', con_cov)
df_sep30 = df_sep30.loc[:, ~df_sep30.columns.duplicated()]
df_sep30

,fips,date,cases,deaths,county,state_code,state,TotalPop,Men,Women,Hispanic,White,Black,Native,Asian,Pacific,VotingAgeCitizen,Income,IncomeErr,IncomePerCap,IncomePerCapErr,Poverty,ChildPoverty,Professional,Service,Office,Construction,Production,Drive,Carpool,Transit,Walk,OtherTransp,WorkAtHome,MeanCommute,Employed,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Unemployment,death_per_1000,cases_per_1000
0,1001,2020-09-30,1791,27,Autauga,AL,Alabama,55036,26899,28137,2.7,75.4,18.9,0.3,0.9,0.0,41016,55317,2838,27824,2024,13.7,20.1,35.3,18.0,23.2,8.1,15.4,86.0,9.6,0.1,0.6,1.3,2.5,25.8,24112,74.1,20.2,5.6,0.1,5.2,0.490588,32.542336
1,1003,2020-09-30,5640,52,Baldwin,AL,Alabama,203360,99527,103833,4.4,83.1,9.5,0.8,0.7,0.0,155376,52562,1348,29364,735,11.8,16.1,35.7,18.2,25.6,9.7,10.8,84.7,7.6,0.1,0.8,1.1,5.6,27.0,89527,80.7,12.9,6.3,0.1,5.5,0.255704,27.734068
2,1005,2020-09-30,896,7,Barbour,AL,Alabama,26201,13976,12225,4.2,45.7,47.8,0.2,0.6,0.0,20269,33368,2551,17561,798,27.2,44.9,25.0,16.8,22.6,11.5,24.1,83.4,11.1,0.3,2.2,1.7,1.3,23.4,8878,74.1,19.1,6.5,0.3,12.4,0.267165,34.197168
3,1007,2020-09-30,664,11,Bibb,AL,Alabama,22580,12251,10329,2.4,74.6,22.0,0.4,0.0,0.0,17662,43404,3431,20911,1889,15.2,26.6,24.4,17.6,19.7,15.9,22.4,86.4,9.5,0.7,0.3,1.7,1.5,30.0,8171,76.0,17.4,6.3,0.3,8.2,0.487157,29.406554
4,1009,2020-09-30,1629,15,Blount,AL,Alabama,57667,28490,29177,9.0,87.4,1.5,0.3,0.1,0.0,42513,47412,2630,22021,850,15.6,25.4,28.5,12.9,23.3,15.8,19.5,86.8,10.2,0.1,0.4,0.4,2.1,35.0,21380,83.9,11.9,4.0,0.1,4.9,0.260114,28.248392
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3136,56037,2020-09-30,339,2,Sweetwater,WY,Wyoming,44527,22981,21546,16.0,79.6,0.8,0.6,0.6,0.5,31025,71083,4336,31700,1368,12.0,15.7,27.7,16.1,20.0,20.8,15.4,77.5,14.4,2.6,2.8,1.3,1.5,20.5,22739,78.4,17.8,3.8,0.0,5.2,0.044917,7.613358
3137,56039,2020-09-30,577,1,Teton,WY,Wyoming,22923,12169,10754,15.0,81.5,0.5,0.3,2.2,0.0,16692,80049,4586,49200,4567,6.8,2.8,39.4,25.4,17.0,11.7,6.5,68.3,6.7,3.8,11.7,3.8,5.7,14.3,14492,82.1,11.4,6.5,0.0,1.3,0.043624,25.171225
3138,56041,2020-09-30,357,2,Uinta,WY,Wyoming,20758,10593,10165,9.1,87.7,0.1,0.9,0.1,0.0,14235,54672,7803,27115,2401,14.9,20.0,30.4,19.4,18.1,16.1,16.1,77.4,14.9,3.3,1.1,1.3,2.0,19.9,9528,71.5,21.5,6.6,0.4,6.4,0.096348,17.198189
3139,56043,2020-09-30,116,6,Washakie,WY,Wyoming,8253,4118,4135,14.2,82.2,0.3,0.4,0.1,0.0,6125,51362,2570,27345,1689,12.8,17.5,32.1,16.3,17.6,18.8,15.3,77.2,10.2,0.0,6.9,1.3,4.4,14.3,3833,69.8,22.0,8.1,0.2,6.1,0.727008,14.055495


In [44]:
df_sep30.to_sql('sep30_table', con_cov, if_exists='replace')


- Find the maximum of highest deaths/cases per 1000 population and their county names 



In [39]:
pd.read_sql_query("SELECT * FROM sep30_table ORDER BY death_per_1000 DESC LIMIT 1", con_cov)

,index,fips,date,cases,deaths,county,state_code,TotalPop,Men,Women,Hispanic,White,Black,Native,Asian,Pacific,VotingAgeCitizen,Income,IncomeErr,IncomePerCap,IncomePerCapErr,Poverty,ChildPoverty,Professional,Service,Office,Construction,Production,Drive,Carpool,Transit,Walk,OtherTransp,WorkAtHome,MeanCommute,Employed,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Unemployment,death_per_1000,cases_per_1000
0,455,13141,2020-09-30,394,42,Hancock,GA,8667,5258,3409,1.8,24.1,72.9,0.0,0.5,0.1,7276,29268,2293,15454,3288,30.0,54.8,25.8,33.0,19.6,6.3,15.3,89.0,5.2,1.0,0.0,0.0,4.7,29.4,2126,66.8,26.8,6.4,0.0,4.0,4.845967,45.45979


In [40]:
pd.read_sql_query("SELECT * FROM sep30_table ORDER BY cases_per_1000 DESC LIMIT 1", con_cov)

,index,fips,date,cases,deaths,county,state_code,TotalPop,Men,Women,Hispanic,White,Black,Native,Asian,Pacific,VotingAgeCitizen,Income,IncomeErr,IncomePerCap,IncomePerCapErr,Poverty,ChildPoverty,Professional,Service,Office,Construction,Production,Drive,Carpool,Transit,Walk,OtherTransp,WorkAtHome,MeanCommute,Employed,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Unemployment,death_per_1000,cases_per_1000
0,2511,47169,2020-09-30,1675,7,Trousdale,TN,8773,4654,4119,1.1,84.9,11.4,0.1,0.3,0.0,6814,41776,9150,22863,2711,13.3,14.8,25.2,20.9,25.4,10.5,17.9,78.0,15.9,0.1,1.1,0.0,4.9,29.2,3834,76.3,16.1,7.6,0.0,8.2,0.797903,190.926707


- Aggregate the data in `sep30_table` at the state level and find the following
  - Calculate the number of total cases/deaths per 1000 population
  - Which state had the highest cases/deaths, how many cases/deaths?


In [48]:
pd.read_sql_query("SELECT state, state_code, SUM(deaths), SUM(cases) FROM sep30_table GROUP BY state", con_cov)

,state,state_code,SUM(deaths),SUM(cases)
0,Alabama,AL,2540,154701
1,Alaska,AK,56,7805
2,Arizona,AZ,5649,218507
3,Arkansas,AR,1369,81786
4,California,CA,15885,819115
5,Colorado,CO,2051,70515
6,Connecticut,CT,4508,57399
7,Delaware,DE,636,20122
8,Florida,FL,14317,704786
9,Georgia,GA,6854,299279


## Merge the presidential election data and explore the relations


### Check the contents of election_2016 table

In this table, pct is the Trump vote percentage

- Check the table content


- Merge the presidential election results with `sep30_table`
- Explore the relations between Trump vote percentage and Covid-19 variables. 